In [4]:
import glob
import os
from random import shuffle
from nltk.tokenize import TreebankWordTokenizer
from gensim.models.keyedvectors import KeyedVectors
from nlpia.loaders import get_data

EMBEDDING_FILE = '/home/muhammed/Documents/nlpia_codes/GoogleNews-vectors-negative300.bin' # from above
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE,limit = 20000, binary=True);


/home/muhammed/anaconda3/lib/python3.7/site-packages/pugnlp/constants.py:87: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
/home/muhammed/anaconda3/lib/python3.7/site-packages/pugnlp/constants.py:137: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
/home/muhammed/anaconda3/lib/python3.7/site-packages/pugnlp/constants.py:159: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
/home/muhammed/anaconda3/lib/python3.7/site-packages/pugnlp/constants.py:167: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly 

In [6]:
def pre_process_data(file_path):
    positive_path = os.path.join(file_path, 'pos')
    negative_path = os.path.join(file_path, 'neg')
    pos_label = 1
    neg_label = 0
    dataset = []

    for filename in glob.glob(os.path.join(positive_path, '*.txt')):
        with open(filename, 'r') as f:
            dataset.append((pos_label, f.read()))
    for filename in glob.glob(os.path.join(negative_path,"*.txt")):
        with open(filename) as f:
            dataset.append((neg_label, f.read()))
    shuffle(dataset)

    return dataset
train_dataset = pre_process_data('/home/muhammed/Documents/nlpia_codes/ch7/aclImdb/train')
test_dataset = pre_process_data('/home/muhammed/Documents/nlpia_codes/ch7/aclImdb/test')
dataset = train_dataset + test_dataset

In [7]:
def tokenize_and_vectorize(dataset):
    tokenizer = TreebankWordTokenizer()
    vectorized_data = []
   
    for sample in dataset:
        tokens = tokenizer.tokenize(sample[1])
        #the document is tokenized into sentences and sentences into tokens
        sample_vecs = []
        for token in tokens:
            try:
                sample_vecs.append(word_vectors[token])
            except:
                pass #No matching token in the Google w2v
        vectorized_data.append(sample_vecs)
    return vectorized_data


def collect_expected(dataset):
    expected = []
    for sample in dataset:
        expected.append(sample[0])
    return expected

In [8]:
def avg_length(data):
    total_len = 0
    for sample in data:
        total_len += len(sample[1])
    return total_len / len(data)


In [9]:

vectorized_data = tokenize_and_vectorize(dataset)
expected = collect_expected(dataset)
split_point = int(len(vectorized_data)*0.8)
x_train = vectorized_data[:split_point]
#note take the length of the vectorized_data because maybe len(vecotized) != len(dataset)
#since we only using limited version of GoogleEmbeddings
y_train = expected[:split_point]

x_test = vectorized_data[split_point:]
y_test = expected[split_point:]



In [6]:
max_len = 100
batch_size = 32
embedding_dims = 300
epochs = 2


In [7]:
def pad_trunc(data,maxlen):
    #pad dataset with zero embedding vectors and truncate the longer one
    new_data = []
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)
    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
    return new_data

In [9]:
import numpy as np
x_train = pad_trunc(x_train, max_len)
x_test = pad_trunc(x_test, max_len)

x_train = np.reshape(x_train, (len(x_train), max_len, embedding_dims))
y_train = np.array(y_train)
x_test = np.reshape(x_test, (len(x_test), max_len, embedding_dims))
y_test = np.array(y_test)

In [9]:
x_train[0];

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, LSTM
num_neurons = 50
model = Sequential()


In [13]:
model.add(LSTM(
            num_neurons, return_sequences = True,
            input_shape = (max_len, embedding_dims)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))

In [14]:
model.compile('rmsprop','binary_crossentropy', metrics=['accuracy'])


2020-05-19 13:45:13,499 WARNING:     tensorflow:323:            new_func From /home/muhammed/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 50)           70200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 50)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5001      
Total params: 75,201
Trainable params: 75,201
Non-trainable params: 0
_________________________________________________________________


In [1]:
model.fit(x_train, y_train,
         batch_size = batch_size,
         epochs = epochs,
         validation_data = (x_test, y_test))

NameError: name 'model' is not defined

In [2]:
def clean_data(data):
    new_data = []
    VALID = 'abcdefghijklmnopqrstuvwxyz0123456789"\'?!.,:; '
    for sample in data:
        new_sample = []
        for char sample[1].lower():
            if char in VALID:
                new_sample.append(char)
            else:
                new_sample.append('UNK')
    return new_data
listed_data = clean_data(dataset)

SyntaxError: invalid syntax (<ipython-input-2-7a0e9eaa5ec6>, line 6)

In [3]:
def char_pad_trunc(data, maxlen = 1500):
    new_dataset = []
    for sample in data:
        if len(sample) > maxlen:
            new_data = sample[:maxlen]
        elif len(sample) < maxlen:
            pads = maxlen - len(sample)
            new_data = sample + ['PAD']*pads
        else:
            new_data = sample
        new_dataset.append(new_data)
    return new_dataset

#this time instead of working with word2vec embedding you are going to work with one_hot vectors to encode the character
# we will have two dictionaries one maps from token into index and the other maps from index into token
def create_dicts(data):
    chars = set()
    for sample in data:
        chars.update(set(sample))
    char_indices = dict((c,i) for i, c in enumerate(chars))
    indices_char = dict((i,c) for i, c in enumerate(chars))
    return char_indices, indices_char

import numpy as np

def onehot_encoding(dataset, char_indices, maxlen  =1500):
    X = np.zeros((len(dataset), maxlen, len(char_indices.keys())))
    for i, sentence in enumerate(dataset):
        for t, char in enumerate(sentence):
            X[i,t, char_indices[char]] = 1
    return X
#onehot_encoding numpy array of length equal to the number of data samples, each sample with length maxlen
#and each token will be one-hot encoded vector of length equal to the number of characters



In [ ]:
listed_data = clean_data(dataset)
#this step the dataset will be converted from word tokens into char tokens
common_length_data = char_pad_trunc(listed_data, maxlen = 1500)
#here the data will be padded
char_indices, indices_char = create_dicts(common_length_data)
#dictionary for char_indices, indices_char used in encoding the char tokens
encoded_data = onehot_encode(common_length_data)
#numpy array with lenght equals to the total samples of the dataset, each with length maxlen and each char of the data with lenght equal to the number of tokens
split_point = int(len(encoded_data)*0.8)

x_train = encoded_data[:split_point]
x_test = encoded_data[:split_point]
y_train = expected[:split_point]
y_test = expected[split_point:]




In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Flatten, Dense, Dropout

model = Sequential()
model.add(LSTM(num_neurons,
              return_sequences = True,
              input_shape = (maxlen, len(char_indices.keys()))))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation = 'sigmoid'))
model.compile('rmsprop', 'binary_crossentropy', metrics = ['accuarcy'])
model.summary()